In [149]:
import os
import torch
import random
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import shutil
import time
import xml.etree.ElementTree as et
import splitfolders
import wget
import requests
import pickle

from io import BytesIO
from zipfile import ZipFile
import tensorflow as tf
from tqdm import tqdm
from PIL import Image
from torchvision import models
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
BATCH_SIZE = 32

In [2]:
use_gpu = torch.cuda.is_available()
device = 'cuda' if use_gpu else 'cpu'
print('Connected device:', device)

Connected device: cpu


**Загружаем датасет**

In [3]:
# перенесем в папку data наш датасет
source_dir = "C:/Users/илья/Desktop/MIPT/Хакатон/Project/Грибы"
target_dir = 'data/Грибы/'
folder_names = os.listdir(source_dir)
for folder_name in folder_names:
    shutil.move(os.path.join(source_dir, folder_name), target_dir)

# os.mkdir('data')
# для загрузки с гугл диска 
# url = 'https://drive.google.com/uc?export=download&id=120xqh0mYtYZ1Qh7vr-XFzjPbSKivLJjA'
# file_name = wget.download(url, 'data/')

# with ZipFile(file_name, 'r') as zip_file:
#     zip_file.extractall()

# link_lst = [eatable, poisonous]
# for link in link_lst:
#     wget.download(link, 'data/')

****Разделим данные на train, test, valid****

In [142]:
# определяем путь к папке с исходными файлами
input_folder = "data/Грибы" # 
 
# разбиваем датасет на папки с процентным соотношением числа файлов
splitfolders.ratio(input_folder, 'data/Грибы/mooshrooms_splited', ratio = (0.7, 0.15, 0.15), seed=13, group_prefix=None)


Copying files: 0 files [00:00, ? files/s]

Copying files: 2639 files [01:11, 37.09 files/s] 


In [143]:
# Training dataset.
train_dataset = ImageFolder(
    root = "data/Грибы/mooshrooms_splited/train"
)
# Validation dataset.
valid_dataset = ImageFolder(
    root = 'data/Грибы/mooshrooms_splited/val'
)
# Test dataset
test_dataset = ImageFolder(
    root = 'data/Грибы/mooshrooms_splited/test'
)

**Произведем нормализацию и аугментацию - автовыравнивание изображений и автоконтраста (чтобы улучшить обобщающую способность моделей и внизить риск переобучения).**

In [144]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
train_dataset.transform = transforms.Compose([
    transforms.Resize([80, 80]),
    transforms.RandomHorizontalFlip(), # augmentations
    transforms.RandomAutocontrast(), # augmentations
    transforms.RandomEqualize(), # augmentations
    transforms.ToTensor(),
    normalize
])

valid_dataset.transform = transforms.Compose([
    transforms.Resize([80, 80]),
    transforms.ToTensor(),
    normalize
])

test_dataset.transform = transforms.Compose([
    transforms.Resize([80, 80]),
    transforms.ToTensor(),
    normalize
])


In [145]:
# Training data loaders.
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE,
    shuffle=True
)
# Validation data loaders.
valid_loader = DataLoader(
    valid_dataset, batch_size=BATCH_SIZE,
    shuffle=False
)
# Test data loaders
test_loader = DataLoader(
    valid_dataset, batch_size=BATCH_SIZE,
    shuffle=False
)

Возьмём предобученную нейросеть GoogLeNet.

Добавим один слой с выходным параметром out_feature=2 

Замораживаем все слои, кроме последних двух.

In [137]:
def google(): # pretrained=True для tensorflow
    model = models.googlenet(weights=models.GoogLeNet_Weights.IMAGENET1K_V1)
    model.fc = torch.nn.Linear(1024, len(train_dataset.classes))
    
    for param in model.parameters():
        param.requires_grad = True
    model.inception3a.requires_grad = False
    model.inception3b.requires_grad = False
    model.inception4a.requires_grad = False
    model.inception4b.requires_grad = False
    model.inception4c.requires_grad = False
    model.inception4d.requires_grad = False
    model.inception4e.requires_grad = False
    return model

**Определяем функцию обучения модели.**

In [146]:
def train(model, optimizer, train_loader, val_loader, epoch=10):
    loss_train, acc_train = [], []
    loss_valid, acc_valid = [], []
    count = 0
    for epoch in tqdm(range(epoch)):
        losses, equals = [], []
        torch.set_grad_enabled(True)

        # Train.
        model.train()
        for i, (image, target) in enumerate(train_loader):
            image = image.to(device)
            target = target.to(device)
            output = model(image)
            loss = criterion(output,target)

            losses.append(loss.item())
            equals.extend(
                [x.item() for x in torch.argmax(output, 1) == target])

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        loss_train.append(np.mean(losses))
        acc_train.append(np.mean(equals))
        losses, equals = [], []
        torch.set_grad_enabled(False)

        # Validate.
        model.eval()
        for i , (image, target) in enumerate(valid_loader):
            image = image.to(device)
            target = target.to(device)

            output = model(image)
            loss = criterion(output,target)

            losses.append(loss.item())
            equals.extend(
                [y.item() for y in torch.argmax(output, 1) == target])

        loss_valid.append(np.mean(losses))
        acc_valid.append(np.mean(equals))

    return loss_train, acc_train, loss_valid, acc_valid

In [147]:
criterion = torch.nn.CrossEntropyLoss()
criterion = criterion.to(device)

model = google() 
print('Model: GoogLeNet\n')

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
model = model.to(device)

loss_train, acc_train, loss_valid, acc_valid = train(
model, optimizer, train_loader, valid_loader)
print('acc_train:', acc_train, '\nacc_valid:', acc_valid)
print('loss_train:', loss_train, '\nloss_valid:', loss_valid)
# del model
# torch.cuda.empty_cache()

Model: GoogLeNet



100%|██████████| 10/10 [10:47:23<00:00, 3884.30s/it]  

acc_train: [0.613759479956663, 0.7264355362946913, 0.8109425785482124, 0.8499458288190682, 0.9111592632719393, 0.9295774647887324, 0.9517876489707475, 0.9609967497291441, 0.9604550379198267, 0.9777898158179849] 
acc_valid: [0.6911392405063291, 0.7291139240506329, 0.7265822784810126, 0.7468354430379747, 0.7468354430379747, 0.7443037974683544, 0.7443037974683544, 0.7443037974683544, 0.7265822784810126, 0.759493670886076]
loss_train: [0.7829130846878578, 0.5669744549126461, 0.4466885641731065, 0.3582612206709796, 0.2557974136595068, 0.20113099944488755, 0.16277205770642594, 0.12011724559525991, 0.10785711479598079, 0.07243275690566876] 
loss_valid: [0.6753063729176154, 0.5985233531548426, 0.5991730506603534, 0.5612080234747666, 0.6065942232425396, 0.6502542644739151, 0.6412237217793098, 0.6823313396710616, 0.7287287230675037, 0.7676346531281104]


**Проверяем точность модели**

In [148]:
correct = 0
total = 0
with torch.no_grad():
   for data in test_loader:
     images, labels = data
    #  print(images)
    #  print(f"Label: {labels.size(0)}")
     outputs = model(images)
     _, predicted = torch.max(outputs.data, 1)
    #  print(f"foft max: {torch.max(outputs.data, 1)}")
     total += labels.size(0)
     correct += (predicted == labels).sum().item()
     
print(f'Model accuracy: {100 * correct // total} %')

Model accuracy: 75 %


**Работоспособность модели**

In [157]:

url = "https://avatars.mds.yandex.net/i?id=08ecf40616c96615f057052aa310bbb1ac90582d-4143030-images-thumbs&n=13"

# функция, приводящая изображение в нужный для модели формат
def to_prepaate_img(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))

    # load image in RGB mode (png files contains additional alpha channel)
    img = img.convert('RGB')

    # set up transformation to resize the image
    resize = transforms.Resize([70, 70])
    img = resize(img)
    to_tensor = transforms.ToTensor()

    # apply transformation and convert to Pytorch tensor
    img_tenzor = to_tensor(img)
    # torch.Size([3, 70, 70])
    img_norm = normalize(img_tenzor)
    # to_normalize = transforms.Normalize()

    # add another dimension at the front to get NCHW shape
    tensor = img_tenzor.unsqueeze(0)
    # torch.Size([1, 3, 70, 70])
    return tensor

image = to_prepaate_img(url)
outputs = model(image)
_, predicted = torch.max(outputs.data, 1)
classes = { 
           "tensor(0)": "eatble", 
           "tensor(1)": "not eatble"
           }
# print(classes[tensor[0]])
print(classes[str(predicted[0])])
# img

eatble


In [159]:
#  сохраняем модель в объект pkl
pickle.dump(model, open("model.pkl", "wb"))